In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
# GET - запрос
url = 'https://www.kinoafisha.info/rating/movies/' 
page = requests.get(url)

In [3]:
page.status_code

200

In [4]:
soup = bs(page.text, 'html.parser')

In [5]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n<head>\n\t<meta name="revisit-after" content="1 day">\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <script>\n    re = /"hash":"watch",.*"trailerId":"(?<hash>[a-f0-9]{32})"/;\n    m = decodeURI(window.location.hash).match(re);\n    if(m && m.groups.hash){\n      var meta = document.createElement(\'meta\');\n      meta.httpEquiv = "refresh";\n      meta.content = "0; url=https://www.kinoafisha.info/trailers/"+m.groups.hash+"/";\n      document.getElementsByTagName(\'head\')[0].appendChild(meta);\n    }\n    </script>\n\t<title>Топ 1000 лучших фильмов | Киноафиша</title>\n\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <script>if(screen.width>=759) document.currentScript.previousElementSibling.setAttribute(\'content\', \'width=990, initial-scale=1\');</script>\n\n\t<style>\n        :root{\n            --ads-placeholder:\'Реклама\';\n        }\n        :root{--desktop:759px;--smal

In [6]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="1 day" name="revisit-after"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script>
    re = /"hash":"watch",.*"trailerId":"(?<hash>[a-f0-9]{32})"/;
    m = decodeURI(window.location.hash).match(re);
    if(m && m.groups.hash){
      var meta = document.createElement('meta');
      meta.httpEquiv = "refresh";
      meta.content = "0; url=https://www.kinoafisha.info/trailers/"+m.groups.hash+"/";
      document.getElementsByTagName('head')[0].appendChild(meta);
    }
    </script>
<title>Топ 1000 лучших фильмов | Киноафиша</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<script>if(screen.width>=759) document.currentScript.previousElementSibling.setAttribute('content', 'width=990, initial-scale=1');</script>
<style>
        :root{
            --ads-placeholder:'Реклама';
        }
        :root{--desktop:759px;--small-columns:1280px;--normal-columns:1325px;--wide-grid:13

In [7]:
result_list = {
    'name': [],
    'rating': [],
    'description': [],
    'date': [],
    'time': []
}

In [8]:
pagenum = 0
for i in range(3):  
    url = 'https://www.kinoafisha.info/rating/movies/?page=' + str(pagenum) 
    page = requests.get(url)     
    soup = bs(page.text, 'html.parser')
    names = soup.find_all('a', class_='movieItem_title')

    for name in names:
        # Получаем название фильма
        if name is not None:
            result_list['name'].append(name.text)
        else:
            result_list['name'].append(None)

        # Переход на страницу фильма
        film_url = name.get('href')
        if film_url:
            try:
                film_page = requests.get(film_url)
                film_page.raise_for_status()  # Проверка на ошибки HTTP
            except requests.exceptions.RequestException as e:
                continue  # Пропустить текущий фильм

            film_soup = bs(film_page.text, 'html.parser')
            
             # Получаем рейтинг для текущего фильма
            rating = film_soup.find('div', class_='ratingBlockCard_local')
            if rating is None:
                result_list['rating'].append('0')  # Если рейтинг не найден
            else:
                result_list['rating'].append(rating.text.strip())  # Добавляем рейтинг
                
            # Получаем описание фильма
            desc_meta = film_soup.find('div', class_='visualEditorInsertion filmDesc_editor more_content')
            if desc_meta is not None:
                description = desc_meta.text.strip()
            else:
                description = None

            # Получаем продолжительность фильма и дату премьеры
            time_element = None
            date_element = None

            # Ищем все элементы с клаcca
            info_items = film_soup.find_all('div', class_='newFilmInfo_infoItem')
            for i in info_items:
                info_name = i.find('span', class_='newFilmInfo_infoName')
                if info_name:
                    if 'Продолжительность' in info_name.text:
                        time_element = i.find('span', class_='newFilmInfo_infoData')
                    elif 'Премьера в мире' in info_name.text:
                        date_element = i.find('span', class_='newFilmInfo_infoData')

            # Записываем продолжительность и дату
            result_list['time'].append(time_element.text.strip() if time_element else None)
            result_list['date'].append(date_element.text.strip() if date_element else None)
            result_list['description'].append(description)
        else:
            # Если страница фильма недоступна, добавляем None для всех полей
            result_list['description'].append(None)
            result_list['date'].append(None)
            result_list['time'].append(None)
      
    pagenum += 1

# Вывод результата
print(result_list)

{'name': ['Атака титанов: Последняя атака', 'Побег из Шоушенка', 'Зеленая миля', 'Поднятие уровня в одиночку: Второе пробуждение', 'SUGA | Agust D TOUR ‘D-DAY’ THE MOVIE', 'Властелин Колец: Братство Кольца', 'Крестный отец', 'Дикий робот', 'Начало', 'Дюна: Часть вторая', 'Темный рыцарь', 'Человек-Паук: Паутина Вселенных', 'Чон Кук: Я Все еще', 'Унесенные призраками', 'Форрест Гамп', 'Irklais per Atlanta', 'Властелин Колец: Две крепости', '1+1', 'Король Лев', 'Терминатор 2: судный день', 'Криминальное чтиво', 'Властелин Колец: Возвращение Короля', 'Матрица', 'Suga: Road to D-Day', 'Оппенгеймер', 'Мой мир SHINee', 'Лига справедливости Зака Снайдера', 'Бойцовский клуб', 'Приключения Шерлока Холмса и доктора Ватсона: Собака Баскервилей', 'Человек-паук: Нет пути домой', 'Ну, погоди!', 'Леон', 'Назад в будущее', 'Бэтмен: начало', 'Список Шиндлера', 'Молчание ягнят', 'Гладиатор', 'Стражи Галактики. Часть 3', 'Крестный отец 2', 'Ходячий замок', 'BTS: Yet to Come in Cinemas', 'Мстители: Война б

In [9]:
result_list

{'name': ['Атака титанов: Последняя атака',
  'Побег из Шоушенка',
  'Зеленая миля',
  'Поднятие уровня в одиночку: Второе пробуждение',
  'SUGA | Agust D TOUR ‘D-DAY’ THE MOVIE',
  'Властелин Колец: Братство Кольца',
  'Крестный отец',
  'Дикий робот',
  'Начало',
  'Дюна: Часть вторая',
  'Темный рыцарь',
  'Человек-Паук: Паутина Вселенных',
  'Чон Кук: Я Все еще',
  'Унесенные призраками',
  'Форрест Гамп',
  'Irklais per Atlanta',
  'Властелин Колец: Две крепости',
  '1+1',
  'Король Лев',
  'Терминатор 2: судный день',
  'Криминальное чтиво',
  'Властелин Колец: Возвращение Короля',
  'Матрица',
  'Suga: Road to D-Day',
  'Оппенгеймер',
  'Мой мир SHINee',
  'Лига справедливости Зака Снайдера',
  'Бойцовский клуб',
  'Приключения Шерлока Холмса и доктора Ватсона: Собака Баскервилей',
  'Человек-паук: Нет пути домой',
  'Ну, погоди!',
  'Леон',
  'Назад в будущее',
  'Бэтмен: начало',
  'Список Шиндлера',
  'Молчание ягнят',
  'Гладиатор',
  'Стражи Галактики. Часть 3',
  'Крестный

In [10]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
name - 0
rating - 0
description - 9
date - 4
time - 0


### Сохранение данных

In [11]:
file_name = 'Films.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [12]:
df.head(25)

,name,rating,description,date,time
0,Атака титанов: Последняя атака,9.3,После катастрофических действий Эрена его друз...,8 ноября 2024,2 часа 25 минут
1,Побег из Шоушенка,9.1,Успешный банковский служащий Энди Дюфрейн осуж...,10 сентября 1994,2 часа 22 минуты
2,Зеленая миля,8.9,В Луизиане в 1935 году заключенные исправитель...,6 декабря 1999,3 часа 8 минут
3,Поднятие уровня в одиночку: Второе пробуждение,8.9,Спустя более десяти лет после появления «врат»...,26 ноября 2024,1 час 56 минут
4,SUGA | Agust D TOUR ‘D-DAY’ THE MOVIE,8.8,None,8 апреля 2024,1 час 24 минуты
5,Властелин Колец: Братство Кольца,8.8,"Средиземье — континент, населенный разными и о...",10 декабря 2001,2 часа 58 минут
6,Крестный отец,8.8,В центре культовой криминальной драмы Фрэнсиса...,14 марта 1972,2 часа 55 минут
7,Дикий робот,8.8,Робот ROZZUM 7134 (сокращенно «Роз») терпит ко...,7 марта 2024,1 час 41 минута
8,Начало,8.8,"Главный герой фильма, Дом Кобб, владеет редкой...",8 июля 2010,2 часа 28 минут
9,Дюна: Часть вторая,8.7,5 причин посмотреть фильм «Дюна 2»\n«Дюна 2» —...,6 февраля 2024,2 часа 46 минут


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         300 non-null    object
 1   rating       300 non-null    object
 2   description  291 non-null    object
 3   date         296 non-null    object
 4   time         300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB
